In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
images=os.path.join('../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images')
annotations=os.path.join('../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/annotations')
submission=pd.read_csv('../input/face-mask-detection-dataset/submission.csv')
train=pd.read_csv('../input/face-mask-detection-dataset/train.csv')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import cv2
from sklearn.model_selection import train_test_split
import tensorflow 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D
from keras.optimizers import Adam

In [ ]:
options=['face_with_mask','face_no_mask']
train=train[train['classname'].isin(options)]

In [ ]:
train.head()

In [ ]:
a=os.listdir(images)
a.sort()

In [ ]:
b=os.listdir(images)
b.sort()

In [ ]:
train['name'].min()

In [ ]:
train_images=a[1698:]

In [ ]:
image=plt.imread(os.path.join(images,train_images[0]))

In [ ]:
fig,ax=plt.subplots(1)
patch=matplotlib.patches.Rectangle((451,186),444,511,linewidth=2,edgecolor='r',facecolor='none')
ax.imshow(image)
ax.add_patch(patch)

In [ ]:
bbox=[]
for i in range(len(train)):
    arr=[]
    for j in train.iloc[i][['x1','x2','y1','y2']]:
        arr.append(j)
    bbox.append(arr)
train["bbox"]=bbox
def get_boxes(id):
    boxes=[]
    for i in train[train["name"]==str(id)]["bbox"]:
        boxes.append(i)
    return boxes
boxes=get_boxes('1801.jpg')
for i in boxes:
    patch=matplotlib.patches.Rectangle((i[0],i[1]),i[2]-i[0],i[3]-i[1],linewidth=2,edgecolor='r',facecolor='none')
fig,ax=plt.subplots(1)
img=plt.imread(os.path.join(images,train_images[0]))
ax.imshow(img)
ax.add_patch(patch)
    

In [ ]:
data=[]
imgsize=50
for i in range(len(train)):
    arr=[]
    for j in train.iloc[i]:
        arr.append(j)
    img=cv2.imread(os.path.join(images,arr[0]),cv2.IMREAD_GRAYSCALE) 
    cropimg=img[arr[2]:arr[4],arr[1]:arr[3]]
    new_img=cv2.resize(cropimg,(imgsize,imgsize))
    data.append([new_img,arr[5]])
    

In [ ]:
x=[]
y=[]
for i,j in data:
    x.append(i)
    y.append(j)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab=LabelEncoder()
y=lab.fit_transform(y)

In [ ]:
x=np.array(x).reshape(-1,50,50,1)

In [ ]:
x=tensorflow.keras.utils.normalize(x,axis=1)
from keras.utils import to_categorical
y=to_categorical(y)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
x_train.shape

In [ ]:
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [ ]:
from keras.layers import LSTM
model=Sequential()
model.add(Conv2D(100,(3,3),input_shape=x.shape[1:],activation='relu',strides=2))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))
opt = tensorflow.keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=40,batch_size=5)

In [ ]:
test_images=a[:1698]

In [ ]:
testing=cv2.imread(os.path.join(images,test_images[0]),cv2.IMREAD_GRAYSCALE)
testing=cv2.resize(testing,(50,50))

In [ ]:
testing.shape

In [ ]:
testing=testing.reshape(-1,50,50,1)

In [ ]:
model.predict_classes(testing)

In [ ]:
plt.imshow(plt.imread(os.path.join(images,test_images[0])))